# COGS 108 - Final Project 

# Overview

*Fill in your overview here*

# Names

- Susmitha Kalidindi 
- Alyssa Ashmore 
- Shardul Saiya 
- Siyi Wang


# Group Members IDs

- A13167684
- A12833150
- A13964199
- A53233349

# Research Question

- Question 1: Want to study how strong the correlation is between words used in product reviews with the ratings given for that product
- Question 2: Want to study the relationship between specific language that users use in reviews, the value of the  ratings they give and whether they are “fake” or incentivized reviews

Edit: Mapping the keywords in Amazon review text against the review ratings to see if words can be used to predict ratings.

## Background and Prior Work

Background:

References:
- 1) https://reviewmeta.com/
- 2) 

# Hypothesis


# Dataset(s)

Dataset Name: Consumer Reviews of Amazon Products
Link to the dataset: https://www.kaggle.com/datafiniti/consumer-reviews-of-amazon-products#Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv
Number of observations: 34,000 reviews

This dataset contains a list of over 34,000 reviews of Amazon products like the Kindle, Fire TV, including measures on the 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Part 1 - Review prediction without NLP

# Setup

In [1]:
## Import Data from CSV file here, SEE Project proposal for link to csv file
import pandas as pd 

dfraw = pd.read_csv("../consumer-reviews-of-amazon-products/1429_1.csv", engine = 'python')

# Data Cleaning

Describe your data cleaning steps here.

In [2]:
# Get rid of most unwanted rows, only keep 
# Name, Category, Reviews.DoRecommend, Review.numHelpful, Reviews.ratings, Review.text, Review.title 

df1 = dfraw.drop(['asins', 'brand', 'keys', 'manufacturer', 'reviews.sourceURLs', 'reviews.userCity', 
                  'reviews.userProvince', 'reviews.username', 'reviews.dateAdded', 'reviews.dateSeen', 
                  'reviews.didPurchase' , 'reviews.id', 'id' ], axis=1)

#get rid of rewiews without ratings
df1 = df1.dropna(subset=['reviews.rating', 'reviews.text', 'reviews.text'])

# Now df1 contains:
# name, categories, reviews.date, reviews.doRecommend,
# reviews.numHelpful, reviews.rating, reviews.text, reviews.title

In [3]:
#helper function to filter data
def filter_review(review):
    # Convert to string
    string = str(review)
    
    # Data Filtering
    string = string.replace('.', ' ')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    # string = string.replace('\'', ' ')
    string = string.replace('\"', ' ')
    string = string.replace('/', ' ')
    string = string.replace('(', ' ')
    string = string.replace(')', ' ')
    string = string.replace('!', ' ')
    string = string.replace('?', ' ')
    string = string.replace('=', ' ')
    string = string.replace('+', ' ')
    string = string.replace('-', ' ')
    string = string.replace('�', '')
    string = string.replace('☺️', ' ')
    string = string.replace('&', ' ')
    string = string.replace('$', ' ')
    string = string.replace('*', ' ')
    string = string.replace('%', ' ')

    
    return string

In [4]:
# dictionary to count all unique words in customer reviews
# contains word : frequency
words_dict = {}

# Helper function to count words in a string 
# This function directly modifies the words_dict declared above
def word_frequency_counter(review):
    
    string = filter_review(review)

    # Convert to lower case and split
    my_string = string.lower().split()
    for item in my_string:
        if item in words_dict:
            words_dict[item] += 1
        else:
            words_dict[item] = 1

In [5]:
# dictionary to count raw rating value of each word
rating_dict = {}

#Helper function to add up raw review values for each word and 
def rating_counter(review, rating):
    string = filter_review(review)
    rating = int(rating)
    
    # Convert to lower case and split
    my_string = string.lower().split()
    for item in my_string:
        if item in rating_dict:
            rating_dict[item] = rating_dict[item] + rating
        else:
            rating_dict[item] = rating
    

In [7]:
for review in df1['reviews.text']:
    word_frequency_counter(review)
print(words_dict)
for index, row in df1.iterrows():
    rating_counter(row['reviews.text'], row['reviews.rating'])

{'this': 17658, 'product': 3577, 'so': 6096, 'far': 1017, 'has': 3998, 'not': 5531, 'disappointed': 224, 'my': 17301, 'children': 435, 'love': 6623, 'to': 33654, 'use': 7675, 'it': 30891, 'and': 33768, 'i': 31873, 'like': 3815, 'the': 43953, 'ability': 330, 'monitor': 42, 'control': 715, 'what': 2259, 'content': 444, 'they': 2879, 'see': 560, 'with': 9469, 'ease': 322, 'great': 11846, 'for': 26316, 'beginner': 127, 'or': 3090, 'experienced': 26, 'person': 202, 'bought': 5059, 'as': 6390, 'a': 24878, 'gift': 2028, 'she': 4052, 'loves': 2633, 'inexpensive': 274, 'tablet': 9062, 'him': 446, 'learn': 307, 'on': 8384, 'step': 90, 'up': 3547, 'from': 2618, 'nabi': 22, 'he': 2027, 'was': 6476, 'thrilled': 69, 'how': 1304, 'skype': 32, 'already': 463, "i've": 768, 'had': 2925, 'fire': 4812, 'hd': 465, '8': 516, 'two': 750, 'weeks': 213, 'now': 1512, 'is': 20762, 'value': 536, 'we': 3546, 'are': 3732, 'prime': 1379, 'members': 138, 'that': 7536, 'where': 320, 'shines': 8, 'being': 671, 'able': 

In [8]:
# TODO
# Visualize word frequency's for top words with a histogram or some other graph
sorted_words_list = list(sorted(words_dict.items(), key=lambda item: item[1], reverse=True))
words_list_20 = sorted_words_list[0:20] 
df_words = pd.DataFrame(words_list_20, columns=['word', 'frequency'])
df_words.plot(kind='bar', x='word')

In [9]:
# Now we have two dictionaries
# word_dict contains words associated with their frequency in all the ratings
# rating_dict_raw cotains words associated with their raw rating values, summed up


# Now we want to divide the values in rating_dict by the frequency values in word_dict
# Thats what this helper function does, and saves it in rating_dict
def ratings_avg(word_dict, rating_dict):
    for item in word_dict:
        if item in rating_dict:
            rating_dict[item] = rating_dict[item]/word_dict[item]
        else:
            print('This shouldnt happen !!')
    

In [10]:
ratings_avg(words_dict, rating_dict)
print(rating_dict)

{'this': 4.5546494506739155, 'product': 4.563041655018171, 'so': 4.539698162729659, 'far': 4.540806293018682, 'has': 4.589294647323662, 'not': 4.082625203399024, 'disappointed': 3.736607142857143, 'my': 4.622565169643373, 'children': 4.462068965517242, 'love': 4.812018722633248, 'to': 4.483062934569442, 'use': 4.601693811074918, 'it': 4.463921530542876, 'and': 4.583659085524757, 'i': 4.506321965299784, 'like': 4.402883355176933, 'the': 4.477532819147726, 'ability': 4.6, 'monitor': 4.642857142857143, 'control': 4.593006993006993, 'what': 4.432934926958831, 'content': 4.4774774774774775, 'they': 4.442167419242793, 'see': 4.3464285714285715, 'with': 4.515154715387053, 'ease': 4.695652173913044, 'great': 4.684028364004727, 'for': 4.520975832193343, 'beginner': 4.3858267716535435, 'or': 4.402912621359223, 'experienced': 4.615384615384615, 'person': 4.396039603960396, 'bought': 4.558410753113264, 'as': 4.474647887323943, 'a': 4.47588230565158, 'gift': 4.648422090729783, 'she': 4.677690029615

In [11]:
# TODO
# Visualize these average rating values with words


# PART 2
## Detecting Bad customer reviews with NLP

For each textual review, we want to predict if it corresponds to a good review (the customer is happy) or to a bad one (the customer is not satisfied). The reviews overall ratings can range from 0/5 to 5/5. 

We will split those into two categories:

bad reviews have overall ratings < 2.5
good reviews have overall ratings >= 2.5

Then we see if we are be able to predict this information using only the raw textual data from the review. Let’s get it started!

In [58]:
df2 = dfraw.drop(['asins', 'brand', 'keys', 'manufacturer', 'reviews.sourceURLs', 'reviews.userCity', 
                  'reviews.userProvince', 'reviews.username', 'reviews.dateAdded', 'reviews.dateSeen', 
                  'reviews.didPurchase' , 'reviews.id', 'id' ], axis=1)
reviews_df = df2.drop(['name','reviews.date', 'categories'], axis=1)

# create the label
reviews_df["is_bad_review"] = reviews_df["reviews.rating"].apply(lambda x: 1 if x <= 3 else 0)
# select only relevant columns
reviews_df = reviews_df[["reviews.text", "is_bad_review"]]


In [59]:
# This is what our data now looks like
reviews_df.head()

,reviews.text,is_bad_review
0,This product so far has not disappointed. My c...,0
1,great for beginner or experienced person. Boug...,0
2,Inexpensive tablet for him to use and learn on...,0
3,I've had my Fire HD 8 two weeks now and I love...,0
4,I bought this for my grand daughter when she c...,0


In [60]:
# Sample the data to speed up computation,
# uncomment is data is taking too long but shouldnt be necessary
reviews_df = reviews_df.sample(frac = 0.1, replace = False, random_state=42)


# Data cleaning using nltk
### Does the following things:
- lower case the text
- tokenize the text 
- remove punctuation
- remove useless words that contain numbers
- remove useless stop words like ‘the’, ‘a’ ,’this’ etc.
- Part-Of-Speech (POS) tagging: assign a tag to every word to define if it corresponds to a noun, a verb etc. using the WordNet lexical database
- lemmatize the text: transform every word into their root form (e.g. rooms -> room, slept -> sleep)

In [61]:
# return the wordnet object value corresponding to the POS tag
from nltk.corpus import wordnet

def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')

# clean text data

[nltk_data] Downloading package wordnet to /Users/ssaiya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [62]:
def clean_text(text):
    
    # float/int etc to string
    text = str(text)
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    pos_tags = pos_tag(text)
    # lemmatize text
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)

In [63]:
reviews_df["review_clean"] = reviews_df["reviews.text"].apply(lambda x: clean_text(x))
reviews_df.head()

,reviews.text,is_bad_review,review_clean
1763,I received this as a gift after my original Ki...,0,receive gift original kindle fire start give t...
20879,Bought this for easier reading and it meets my...,0,buy easy read meet expectation great display f...
26427,Alexa does all that was described. It is like ...,0,alexa describe like assistant selects music ma...
8344,I researched about tablets and based on my int...,0,research tablet base interest choose amazon fi...
34279,This is not our first Amazon Fire---we have th...,0,first amazon fire---we every tv house cord cut...


# Data Analysis & Results

Include cells that describe the steps in your data analysis.

In [ ]:
# Now rating_dict has what we need to go ahead


# Ethics & Privacy

*Fill in your ethics & privacy discussion here*

# Conclusion & Discussion

*Fill in your discussion information here*